In [46]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

def std_crawler(url):
    driver = webdriver.Chrome(executable_path=r"C:\Users\gihok\chatbot\chromedriver.exe")
    driver.get(url)
    time.sleep(3)
    
    df = pd.DataFrame(columns=['표준번호','표준명(한글)','대체 표준','표준명(영문)','KS원문보기','표준분야','표준구분','제정일','최종개정확인일',
                               '기술심의회','전문위원회','적용범위','표준개발협력기관','담당부처','담당부서','담당자명',
                               '담당자 연락처','국제표준 부합화','인용표준','기술기준','ICS Code','표준이력사항','인증심사기준'])
    while True:
        try:
            # 페이지 넘기기
            # for i in range(3, 14):
            for i in range(3,6):
                driver.find_element(By.XPATH, '//*[@id="tabs-container"]/div[3]/div/div/ul/li[{}]/a'.format(i)).click()
                
                time.sleep(3)
                
                # 표준 들어가기
                for j in range(1, 11):
                    driver.find_element(By.XPATH, '//*[@id="tabs-container"]/div[2]/table/tbody/tr[{}]/td[2]/a'.format(j)).click()
                    
                    time.sleep(3)
                    
                    #crawling 작업
                    try:
                        dict = {}
                        for z in range(1, 22):
                            # 인용표준 및 기술기준
                            if z == 18:
                                title_1 = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[2]/div[1]/table/tbody/tr[{}]/th[1]'.format(z)).text
                                content_1 = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[2]/div[1]/table/tbody/tr[18]/td[1]').text
                                title_2 = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[2]/div[1]/table/tbody/tr[{}]/th[2]'.format(z)).text
                                content_2 = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[2]/div[1]/table/tbody/tr[18]/td[2]').text
                                dict[title_1] = [content_1]
                                dict[title_2] = [content_2]
                                
                            else:    
                                title = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[2]/div[1]/table/tbody/tr[{}]/th[1]'.format(z)).text
                                content = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[2]/div[1]/table/tbody/tr[{}]/td[1]'.format(z)).text
                                dict[title] = [content]
                        dict_df = pd.DataFrame(dict)
                        df = pd.concat([df, dict_df], ignore_index=True)
                        
                    except:
                        break
                    
                    driver.back()
                    driver.refresh()
                    time.sleep(3)
                return df
        except:
            break
        
    return df

In [47]:
df = std_crawler("https://standard.go.kr/KSCI/standardIntro/getStandardSearchList.do?menuId=919&topMenuId=502")

C:\Users\gihok\AppData\Local\Temp\ipykernel_115620\3869950124.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:\Users\gihok\chatbot\chromedriver.exe")


In [48]:
df

,표준번호,표준명(한글),대체 표준,표준명(영문),KS원문보기,표준분야,표준구분,제정일,최종개정확인일,기술심의회,...,담당부처,담당부서,담당자명,담당자 연락처,국제표준 부합화,인용표준,기술기준,ICS Code,표준이력사항,인증심사기준
0,KS A 0001,표준서의 서식 및 작성방법,NaN,Rules for the drafting and presentation of Kor...,표준 원문보기,(A) 기본 - (A10) 기본일반,방법,1962-06-26,2021-01-11,기계기본 기술심의회(B),...,국가기술표준원,산업표준혁신과,고정하,--,대응국제표준 부합화수준\n해당없음,147 건,1 건,01.120 (표준화. 일반 규칙),"변경일자 구분 고시번호 제정,개정,폐지 사유 신구대비표\n1962-06-26 제정 ...",제/개정일자 구분 첨부파일\n관련 자료가 없습니다.
1,KS A 0002 (폐지),법정계량 용어 (폐지),,International vocabulary of legal metrology,NaN,(A) 기본 - (A17) 기타,전달,2004-12-24,2019-12-31,기계기본 기술심의회(B),...,국가기술표준원,기계융합산업표준과,양승배,043-870-5373,대응국제표준 부합화수준\n해당없음,0 건,2 건,해당없음,"변경일자 구분 고시번호 제정,개정,폐지 사유 신구대비표\n2004-12-24 제정 ...",제/개정일자 구분 첨부파일\n관련 자료가 없습니다.
2,KS A 0003 (폐지),법정계량 용어 (폐지),KSA0002,Vocabulary of terms in legal metrology,NaN,(A) 기본 - (A10) 기본일반,전달,2004-12-24,2013-11-05,기계기본 기술심의회(B),...,국가기술표준원,기계융합산업표준과,양승배,043-870-5373,대응국제표준 부합화수준\n관련 자료가 없습니다.,0 건,0 건,01.040.17 (계랑 및 측정. 물리현상 (용어)),"변경일자 구분 고시번호 제정,개정,폐지 사유 신구대비표\n2004-12-24 제정 ...",제/개정일자 구분 첨부파일\n관련 자료가 없습니다.
3,KS A 0004 (폐지),분석 . 시험의 허용차 통칙 (폐지),,GENERAL RULES FOR PERMISSIBLE TOLERANCE OF CHE...,NaN,(A) 기본 -,전달,1981-12-11,2002-12-30,,...,국가기술표준원,기계융합산업표준과,이경희,043-870-5379,대응국제표준 부합화수준\nISO 5725-1 NEQ 동일하지 않음\nISO 5725...,0 건,8 건,,"변경일자 구분 고시번호 제정,개정,폐지 사유 신구대비표\n1981-12-11 개정 ...",제/개정일자 구분 첨부파일\n관련 자료가 없습니다.
4,KS A 0005,제도 ― 통칙(도면 작성의 일반코드),NaN,TECHNICAL DRAWING-GENERAL CODE OF DRAWING PRAC...,표준 원문보기,(A) 기본 - (A10) 기본일반,전달,1969-10-25,2019-12-31,기계기본 기술심의회(B),...,국가기술표준원,기계융합산업표준과,김민우,043-870-5376,대응국제표준 부합화수준\n해당없음,87 건,1 건,01.100.00 (기술도안 일반),"변경일자 구분 고시번호 제정,개정,폐지 사유 신구대비표\n1969-10-25 확인 ...",제/개정일자 구분 첨부파일\n관련 자료가 없습니다.
5,KS A 0006,시험 장소의 표준 상태,NaN,STANDARD ATMOSPHERIC CONDITIONS FOR TESTING,표준 원문보기,(A) 기본 - (A10) 기본일반,전달,1971-05-22,2019-12-24,기계기본 기술심의회(B),...,국가기술표준원,기계융합산업표준과,김민우,043-870-5376,대응국제표준 부합화수준\nISO 554:1976 IDT 일치,0 건,64 건,19.020 (시험 조건 및 절차 일반),"변경일자 구분 고시번호 제정,개정,폐지 사유 신구대비표\n1971-05-22 확인 ...",제/개정일자 구분 첨부파일\n관련 자료가 없습니다.
6,KS A 0007 (폐지),음향 용어 ( 기기 ) (폐지),,GLOSSARY OF ACOUSTICAL TERMS (TRANSDUCERS AND ...,NaN,(A) 기본 -,전달,1982-11-04,2001-11-19,전기응용 기술심의회(C),...,국가기술표준원,전기전자정보표준과,김성희,043-870-5365,대응국제표준 부합화수준\n관련 자료가 없습니다.,0 건,0 건,01.040.17 (계랑 및 측정. 물리현상 (용어)) 17.140.00 (음향 및...,"변경일자 구분 고시번호 제정,개정,폐지 사유 신구대비표\n1982-11-04 확인 ...",제/개정일자 구분 첨부파일\n관련 자료가 없습니다.
7,KS A 0009 (폐지),음향 용어 ( 음성 . 청각 . 음악 ) (폐지),,GLOSSARY OF ACOUSTICAL TERMS (HEARING.SPEECH.M...,NaN,(A) 기본 -,전달,1982-11-04,2001-11-19,전기응용 기술심의회(C),...,국가기술표준원,전기전자정보표준과,김성희,043-870-5365,대응국제표준 부합화수준\n관련 자료가 없습니다.,0 건,0 건,01.040.17 (계랑 및 측정. 물리현상 (용어)) 17.140.01 (음향 측...,"변경일자 구분 고시번호 제정,개정,폐지 사유 신구대비표\n1982-11-04 확인 ...",제/개정일자 구분 첨부파일\n관련 자료가 없습니다.
8,KS A 0010,오염 물질 조정 용어,NaN,Glossary of terms for contamination control,표준 원문보기,(A) 기본 - (A10) 기본일반,전달,1985-05-08,2020-12-30,산업환경 기술심의회(I),...,국가기술표준원,바이오화학서비스표준과,지민호,043-870-5394,대응국제표준 부합화수준\n해당없음,2 건,0 건,01.040.13 (환경 및 보건. 안전도 (용어)),"변경일자 구분 고시번호 제정,개정,폐지 사유 신구대비표\n1985-05-08 확인 ...",제/개정일자 구분 첨부파일\n관련 자료가 없습니다.
9,KS A 0011,물체색의 색 이름,NaN,Names of non - luminous object colours,표준 원문보기,(A) 기본 - (A10) 기본일반,전달,1967-12-30,2020-12-22,문화 기술심의회(A),...,국가기술표준원,바이오화학서비스표준과,박정은,043-870-5401,대응국제표준 부합화수준\n해당없음,6 건,8 건,17.180.20 (빛의 색 및 측정),"변경일자 구분 고시번호 제정,개정,폐지 사유 신구대비표\n1967-12-30 개정 ...",제/개정일자 구분 첨부파일\n관련 자료가 없습니다.


In [49]:
df.loc[0]

표준번호                                                KS A 0001
표준명(한글)                                        표준서의 서식 및 작성방법
대체 표준                                                     NaN
표준명(영문)     Rules for the drafting and presentation of Kor...
KS원문보기                                                표준 원문보기
표준분야                                      (A) 기본 - (A10) 기본일반
표준구분                                                       방법
제정일                                                1962-06-26
최종개정확인일                                            2021-01-11
기술심의회                                           기계기본 기술심의회(B)
전문위원회                                                 표준기술연구회
적용범위        이 표준은 국가표준(이하, 표준이라 한다.)의 구성 및 표현형식을 규정하며 한국산업...
표준개발협력기관                                               한국표준협회
담당부처                                                  국가기술표준원
담당부서                                                  산업표준혁신과
담당자명                                                      고정하
담당자 연락처 